In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 106.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88

In [2]:
# 1. Install compatible NumPy and Scipy versions
!pip install "numpy<2.0" "scipy>=1.10" --upgrade --quiet

# 2. INSTRUCTION:
# After this cell finishes running, you MUST restart the Kernel.
# Go to the toolbar: Run > Restart Kernel (or click the 'Refresh' arrow button next to the Stop button).
# Then run the Training Cell below.
print("✅ Installation complete. PLEASE RESTART THE KERNEL NOW before running the next cell.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 106.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 56.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
ydata-profiling 4.17.0 requires scipy<1.16,>=1.4.1, but you have scipy 1.16.3 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-cola

In [3]:
import os
import shutil
import yaml
import torch
import glob
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
# Load a model
model = YOLO("yolo11l.pt")  # load a pretrained model (recommended for training)

In [ ]:
# ==========================================
# 1. CONFIGURATION
# ==========================================
PROJECT_NAME = "Forest_Fire_Smoke_Detection"
RUN_NAME = "yolov11_fire_smoke_v2_STABLE" 

# INPUT: Your curated dataset
INPUT_DATASET_DIR = "/kaggle/input/fire-smoke-v2/dataset_split-V2" 

# HYPERPARAMETERS (Tuned for P100 16GB & Night/Cloud Robustness)
IMGSZ = 640          # 640 is standard/stable.
BATCH_SIZE = 16      # Increased for better Batch Normalization stability.
EPOCHS = 150

# ==========================================
# 2. HIGH-SPEED DATA SETUP
# ==========================================
print("🚀 Setting up Industrial Training Environment...")

ROOT_WORKING_DIR = "/kaggle/working/"
DATASET_NAME = INPUT_DATASET_DIR.split('/')[-1] 
DATA_DIR = os.path.join(ROOT_WORKING_DIR, DATASET_NAME)

def setup_data(source_dir, target_dir):
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"❌ Input dataset not found at {source_dir}.")
    
    if os.path.exists(target_dir):
        print(f"ℹ️ Data already exists in {target_dir}. Skipping copy.")
    else:
        print(f"📂 Copying data to local drive for High-Speed I/O... (Please wait)")
        shutil.copytree(source_dir, target_dir)
        print("✅ Data copy complete.")
    return target_dir

# Execute Setup
setup_data(INPUT_DATASET_DIR, DATA_DIR)

# Generate data.yaml
yaml_content = {
    'path': DATA_DIR, 
    'train': 'train/images',  
    'val': 'valid/images',    
    'test': 'test/images',
    'names': {0: 'smoke', 1: 'fire'} 
}

yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f)
print(f"✅ Config saved to {yaml_path}")

# --- Everything below Section 3 has been updated for stability ---
# ==========================================
# 3. ROBUST TRAINING LOOP (FORCED STABILITY)
# ==========================================

# Check for resume
last_ckpt = "/kaggle/working/runs/detect/yolov11_fire_smoke_v1_STABLE/weights/last.pt"
resume_training = os.path.exists(last_ckpt)

if resume_training:
    print(f"♻️ RESUMING training from {last_ckpt}")
    model = YOLO(last_ckpt)
else:
    print("🆕 Starting NEW training session")
    model = YOLO('yolo11l.pt') 

print("🔥 Initiating Training with FORCED AdamW STABILITY MODE...")

try:
    results = model.train(
        data=yaml_path,
        project="/kaggle/working/runs/detect",
        name=RUN_NAME,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH_SIZE,
        
        # --- CRITICAL STABILITY FIXES (FORCED ADAMW) ---
        optimizer='AdamW',         # FORCED: AdamW is stable at low LR
        lr0=0.001,                 # FORCED: Safe, low initial Learning Rate
        cos_lr=True,               # Use Cosine LR scheduler for smooth decay
        warmup_epochs=3.0,
        
        # --- NIGHT MODE & ROBUSTNESS ---
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,                 # High V for simulating low light/glare
        
        # --- CONTEXT & HARD NEGATIVES ---
        mosaic=1.0,                # Critical for learning small smoke/context
        mixup=0.1,
        degrees=10.0,              # For UAV robustness
        fliplr=0.5,
        close_mosaic=10,
        
        # --- SYSTEM ---
        device=0,
        workers=4,
        save=True,
        save_period=5,
        resume=resume_training,
        plots=True
    )
    print("🎉 Training Completed Successfully!")

except Exception as e:
    print(f"⚠️ Training Interrupted: {e}")
    print("To resume, simply re-run this cell.")

🚀 Setting up Industrial Training Environment...
ℹ️ Data already exists in /kaggle/working/dataset_split-V2. Skipping copy.
✅ Config saved to /kaggle/working/data.yaml
🆕 Starting NEW training session
🔥 Initiating Training with FORCED AdamW STABILITY MODE...
Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf

In [3]:
# ==========================================
# 1. CONFIGURATION
# ==========================================
PROJECT_NAME = "Forest_Fire_Smoke_Detection"
RUN_NAME = "yolov11_fire_smoke_v2_STABLE" 

# INPUT: Your curated dataset
INPUT_DATASET_DIR = "/kaggle/input/fire-smoke-v2/dataset_split-V2" 

# HYPERPARAMETERS (Tuned for P100 16GB & Night/Cloud Robustness)
IMGSZ = 640          # 640 is standard/stable.
BATCH_SIZE = 16      # Increased for better Batch Normalization stability.
EPOCHS = 150

# ==========================================
# 2. HIGH-SPEED DATA SETUP
# ==========================================
print("🚀 Setting up Industrial Training Environment...")

ROOT_WORKING_DIR = "/kaggle/working/"
DATASET_NAME = INPUT_DATASET_DIR.split('/')[-1] 
DATA_DIR = os.path.join(ROOT_WORKING_DIR, DATASET_NAME)

def setup_data(source_dir, target_dir):
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"❌ Input dataset not found at {source_dir}.")
    
    if os.path.exists(target_dir):
        print(f"ℹ️ Data already exists in {target_dir}. Skipping copy.")
    else:
        print(f"📂 Copying data to local drive for High-Speed I/O... (Please wait)")
        shutil.copytree(source_dir, target_dir)
        print("✅ Data copy complete.")
    return target_dir

# Execute Setup
setup_data(INPUT_DATASET_DIR, DATA_DIR)

# Generate data.yaml
yaml_content = {
    'path': DATA_DIR, 
    'train': 'train/images',  
    'val': 'valid/images',    
    'test': 'test/images',
    'names': {0: 'smoke', 1: 'fire'} 
}

yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f)
print(f"✅ Config saved to {yaml_path}")

# --- Everything below Section 3 has been updated for stability ---
# ==========================================
# 3. ROBUST TRAINING LOOP (FORCED STABILITY)
# ==========================================

# Check for resume
last_ckpt = "/kaggle/working/runs/detect/yolov11_fire_smoke_v2_STABLE2/weights/last.pt"
resume_training = os.path.exists(last_ckpt)

if resume_training:
    print(f"♻️ RESUMING training from {last_ckpt}")
    model = YOLO(last_ckpt)
else:
    print("🆕 Starting NEW training session")
    model = YOLO('yolo11l.pt') 

print("🔥 Initiating Training with FORCED AdamW STABILITY MODE...")

try:
    results = model.train(
        data=yaml_path,
        project="/kaggle/working/runs/detect",
        name=RUN_NAME,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH_SIZE,
        
        # --- CRITICAL STABILITY FIXES (FORCED ADAMW) ---
        optimizer='AdamW',         # FORCED: AdamW is stable at low LR
        lr0=0.001,                 # FORCED: Safe, low initial Learning Rate
        cos_lr=True,               # Use Cosine LR scheduler for smooth decay
        warmup_epochs=3.0,
        
        # --- NIGHT MODE & ROBUSTNESS ---
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,                 # High V for simulating low light/glare
        
        # --- CONTEXT & HARD NEGATIVES ---
        mosaic=1.0,                # Critical for learning small smoke/context
        mixup=0.1,
        degrees=10.0,              # For UAV robustness
        fliplr=0.5,
        close_mosaic=10,
        
        # --- SYSTEM ---
        device=0,
        workers=4,
        save=True,
        save_period=5,
        resume=resume_training,
        plots=True
    )
    print("🎉 Training Completed Successfully!")

except Exception as e:
    print(f"⚠️ Training Interrupted: {e}")
    print("To resume, simply re-run this cell.")

🚀 Setting up Industrial Training Environment...
ℹ️ Data already exists in /kaggle/working/dataset_split-V2. Skipping copy.
✅ Config saved to /kaggle/working/data.yaml
♻️ RESUMING training from /kaggle/working/runs/detect/yolov11_fire_smoke_v2_STABLE2/weights/last.pt
🔥 Initiating Training with FORCED AdamW STABILITY MODE...
Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=Fa

In [4]:
# ==========================================
# 4. VALIDATION ON TEST SET (ADDED CONF/IOU)
# ==========================================
# This validates specifically on your 'Test' split (which you mentioned creating)
print("\n📊 Running Final Validation on TEST set...")
try:
    # Load best weights
    best_model = YOLO(f"/kaggle/working/runs/detect/yolov11_fire_smoke_v2_STABLE2/weights/best.pt")
    
    # We set a low confidence threshold (conf=0.01) to find ALL possible detections, 
    # ensuring the model isn't mistakenly detecting clouds/glare (your hard negatives).
    metrics = best_model.val(
        split='test', 
        save_json=True, 
        plots=True, 
        conf=0.01, # Low threshold to check for false positives
        iou=0.6    # Standard IoU
    )
    
    print(f"🔥 Map50: {metrics.box.map50}")
    print(f"🔥 Map50-95: {metrics.box.map}")
    
except Exception as e:
    print(f"⚠️ Validation skipped or failed: {e}")


📊 Running Final Validation on TEST set...
Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11l summary (fused): 190 layers, 25,280,854 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 118.8±161.4 MB/s, size: 427.7 KB)
val: Scanning /kaggle/working/dataset_split-V2/test/labels... 1106 images, 141 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1106/1106 576.2it/s 1.9s.1s
val: New cache created: /kaggle/working/dataset_split-V2/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 70/70 2.8it/s 25.4s0.4ss
                   all       1106       1762      0.717       0.68      0.729      0.451
                 smoke        965       1212      0.802      0.739      0.813      0.574
                  fire        281        550      0.633       0.62      0.645      0.327
Speed: 1.0ms preprocess, 12.2ms inference, 0.0ms loss, 0.9ms postproc

In [10]:
!cp /kaggle/input/yaml-file/botsort_wildfire.yaml /kaggle/working/botsort_wildfire.yaml


In [14]:
%%writefile /kaggle/working/botsort_wildfire.yaml
# Wildfire-Optimized BOT-SORT Configuration

tracker_type: botsort

# ---- Thresholds tuned for smoke/fire ----
track_high_thresh: 0.10        # lower → keeps weak smoke detections
track_low_thresh: 0.01         # very low → reconnects faint smoke
new_track_thresh: 0.05         # low → allows new smoke tracks to start early

track_buffer: 50               # higher → smoke disappears and reappears often
match_thresh: 0.65             # slightly lower → smoke shapes vary a lot
fuse_score: True               # helps stabilize flickering smoke

# ---- Motion compensation ----
gmc_method: sparseOptFlow      # BEST for camera shakes and mountain cameras

# ---- ReID OFF (fire/smoke have no identity) ----
with_reid: False
proximity_thresh: 0.5
appearance_thresh: 0.8
model: auto



Overwriting /kaggle/working/botsort_wildfire.yaml


In [8]:
!yolo track \
    model=/kaggle/working/runs/detect/yolov11_fire_smoke_v2_STABLE2/weights/best.pt \
    source="/kaggle/working/2021062016 bm s (Big Black Mountain south).mp4" \
    tracker=/kaggle/working/botsort_wildfire.yaml \
    conf=0.15 \
    iou=0.55 \
    imgsz=640 \
    save=True \
    verbose=False


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.11.13 environment at: /usr
Resolved 2 packages in 315ms
Prepared 1 package in 18ms
Installed 1 package in 4ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11l summary (fused): 190 layers, 25,280,854 parameters, 0 gradients, 86.6 GFLOPs
Results saved to /kaggle/working/runs/detect/track11
💡 Learn more at https://docs.ultralytics.com/modes/track
